In [ ]:
# pip install sec-api
# pip install neo4j
# pip install untangle
# pip install --upgrade pandas --user

In [1]:
from py2neo import Graph, Node, Relationship
from sec_api import QueryApi
import pandas as pd
import http.client
from urllib.request import urlopen
from ipynb.fs.full.Functions_sec import *
import os
#import xmltodict

In [2]:
directory = os.getcwd()
path = directory + '/Data/'

# Fetching Filings of 13f/HR 

In [1056]:
# # initialize the query API
queryApi = QueryApi(api_key="deb4d5052cc76e66ae6b4a1b7b5ad07bcd1a0beda77f3ba69cc0b10457d2f9cb")

Filings=get_13f_filings()
Filings=Filings.iloc[Filings.astype(str).drop_duplicates().index]

In [ ]:
Filings_HR_A=get_13f_HRA_filings()

# Fetching 13FHR/A data 

In [ ]:
Filings_HR_A=get_13f_HRA_filings()
Filings_HR_A=Filings_HR_A.sort_values(by=['filedAt'])

# Web scraping to get 13FHR/A holdings

In [ ]:
Minus=Filings_HR_A[Filings_HR_A.duplicated('cik', keep="last")]
Filings_HR_A=Filings_HR_A.drop(Minus.index).reset_index(drop=True)
url_tbl_final=fetch_url(Filings_HR_A)
hr_a_filings=fetch_data_from_xml(url_tbl_final)

# Reading the CIK lookup data

In [ ]:
with open(path+'cik-lookup-data.txt') as f:
    lines = f.readlines()
txt_file=pd.DataFrame(lines)
txt_file.columns=["cik"]

In [ ]:
df = split_dataframe(txt_file, chunk_size=100000)
cik_final=cik_(df)

# Data Pre-processing for 13FHR/A

In [1137]:
hr_a_filings["Form"]="13FHR/A"
hr_a_filings['value']=hr_a_filings['value']*1000
x=hr_a_filings.groupby(by=['cusip',  'cik','titleOfClass','nameOfIssuer','putCall','investmentDiscretion','Form'],dropna=False).sum()
HRA_final=x.reset_index()

## Extracting 13FHR/A cik's from 13FHR data

In [ ]:
hr_adjustments=Filings[Filings["cik"].isin (HRA_final["cik"])].reset_index(drop=True)
Holdings_adjustments=Extracting_holdings(hr_adjustments)
Holdings_adjustments['Form']='13FHR'

In [ ]:
Holdings_adjustments=Holdings_adjustments.drop(['id','shrsOrPrnAmt.sshPrnamtType','votingAuthority.Shared','votingAuthority.None'],axis=1)
Holding_adjustments_grouped=Holdings_adjustments.groupby(by=['cusip','cik','titleOfClass','nameOfIssuer','putCall','investmentDiscretion','Form'],dropna=False).sum()

In [ ]:
Holding_adjustments_grouped=Holding_adjustments_grouped.reset_index()
Holding_adjustments_grouped.columns=(['cusip', 'cik', 'titleOfClass', 'nameOfIssuer', 'putCall', 'investmentDiscretion', 'Form', 'value', 'shrsOrPrnAmt',  'votingAuthority'])

# Joining the 13F-HR and 13F-HR/A holdings

In [1158]:
Filings_for_holding=Filings.drop(Filings[Filings["cik"].isin(HRA_final["cik"])].index).reset_index(drop=True)
HRA_filings=pd.concat((Holding_adjustments_grouped,HRA_final),axis=0, ignore_index=True)
HRA_filings=HRA_filings.sort_values(by=['cik','cusip','Form'])

In [ ]:
Minus=HRA_filings[HRA_filings.duplicated(['cusip',  'cik','titleOfClass','nameOfIssuer','putCall','investmentDiscretion'], keep='last')]
C=HRA_filings.drop(Minus.index).reset_index(drop=True)
Filings_for_holding=Filings.drop(Filings[Filings["cik"].isin(HRA_final["cik"])].index).reset_index(drop=True)

# Holdings extraction from the joined data

In [ ]:
df=split_dataframe(Filings_for_holding)
Hold_final=Holdings(df)

In [1189]:
Hold_final["Form"]='13FHR'
Hold_final=Hold_final.groupby(by=['cusip',  'cik','titleOfClass','nameOfIssuer','putCall','investmentDiscretion'],dropna=False).sum()
Hold_final=Hold_final.reset_index()
Hold_final=pd.concat((Hold_final,C),axis=0, ignore_index=True)

# Cleaning the CIK lookup data

In [ ]:
cik_final=cik_final.drop_duplicates()

cik_final.columns=["Company","CIK","Separator","se","k"]

for x in range(len(cik_final)):   
    if cik_final["Separator"][x]!="\n":
        cik_final["Company"][x]=cik_final["Company"][x]+cik_final["CIK"][x]
        cik_final["CIK"][x]=cik_final["Separator"][x]
        
h=cik_final[(cik_final["se"].notna()) & (cik_final["se"]!="\n")]

In [227]:
for x in range(len(h)):   
    h.iloc[x]["Company"]=h.iloc[x]["Company"]+h.iloc[x]["CIK"]+h.iloc[x]["Separator"]
    h.iloc[x]["CIK"]=h.iloc[x]["se"]

cik_final=cik_final.drop(h.index)

#cik_final=cik_final.append(h)
cik_final=pd.concat((cik_final, h), axis = 0,ignore_index=True)

cik_final=cik_final.drop(columns=["Separator","se","k"])
cik_final.columns=["nameOfIssuer","CIK"]

cik_final=cik_final.drop_duplicates()
# cik_final["nameOfIssuer"]=cik_final["nameOfIssuer"].str.upper()

# Saving files

In [ ]:
cik_final.to_csv(path+'cik_final.csv',index=None)
Hold_final.to_csv(path+'Hold_final.csv',index=None)
Filings_for_holding.to_csv(path + 'Filing_final.csv', index=None)